In [4]:
import os, pandas as pd
from datetime import date, timedelta
from dotenv import load_dotenv
import finnhub

load_dotenv()
FINNHUB_KEY = os.getenv("FINNHUB_API_KEY")
if not FINNHUB_KEY:
    raise RuntimeError("Missing FINNHUB_API_KEY in .env")

client = finnhub.Client(api_key=FINNHUB_KEY)

def finnhub_company_news(symbol="AAPL", days_back=7):
    end = date.today()
    start = end - timedelta(days=days_back)
    news = client.company_news(symbol, _from=str(start), to=str(end))
    if not news:
        print(f"No company news for {symbol} in last {days_back} days.")
        return pd.DataFrame()
    df = pd.DataFrame([{
        "symbol": symbol,
        "source": n.get("source"),
        "headline": n.get("headline"),
        "summary": n.get("summary"),
        "datetime": pd.to_datetime(n.get("datetime"), unit="s", utc=True),
        "url": n.get("url")
    } for n in news])
    return df.sort_values("datetime").reset_index(drop=True)

df_company = finnhub_company_news("NVDA", days_back=7)
df_company.head()


,symbol,source,headline,summary,datetime,url
0,NVDA,SeekingAlpha,"ETB: A Consistent Buy-Write Options Fund, With...",Eaton Vance Tax-Managed Buy-Write Income Fund ...,2025-10-19 08:05:00+00:00,https://finnhub.io/api/news?id=80e94f4dd49bb60...
1,NVDA,Yahoo,Where Will Nvidia Stock Be in 3 Years?,Nvidia has led the AI arms race for the past t...,2025-10-19 09:37:00+00:00,https://finnhub.io/api/news?id=27276d29dafdfd7...
2,NVDA,Yahoo,"My Favorite AI Growth Stock to Invest $1,000 i...",ASML is well-positioned to benefit from AI chi...,2025-10-19 09:55:00+00:00,https://finnhub.io/api/news?id=738ee528205045c...
3,NVDA,SeekingAlpha,"BE Semiconductor: Expensive Now, But Its AI Ch...",BE Semiconductor leads in die-attach tech for ...,2025-10-19 10:11:31+00:00,https://finnhub.io/api/news?id=06e3ce65f8151ee...
4,NVDA,Yahoo,Is This AI Stock Still Worth Buying After Its ...,Nvidia is the linchpin of the booming AI marke...,2025-10-19 10:30:00+00:00,https://finnhub.io/api/news?id=c5de16206a5be62...


In [5]:
def finnhub_general_news(category="general", limit=20):
    """
    category ∈ {"general","forex","crypto","merger"}
    """
    items = client.general_news(category=category, min_id=0)
    rows = [{
        "category": category,
        "source": x.get("source"),
        "headline": x.get("headline"),
        "summary": x.get("summary"),
        "datetime": pd.to_datetime(x.get("datetime"), unit="s", utc=True),
        "url": x.get("url")
    } for x in items[:limit]]
    return pd.DataFrame(rows).sort_values("datetime").reset_index(drop=True)

df_general = finnhub_general_news("general", limit=10)
df_general.head()


,category,source,headline,summary,datetime,url
0,general,CNBC,American Airlines is arriving late to the luxu...,American Airlines has fallen behind large riva...,2025-10-26 12:03:58+00:00,https://www.cnbc.com/2025/10/26/american-airli...
1,general,MarketWatch,The surprising stocks leading the tech sector ...,Western Digital and Seagate were once sleepy c...,2025-10-26 13:00:00+00:00,https://www.marketwatch.com/story/the-surprisi...
2,general,MarketWatch,"Target might be cutting jobs, but its ‘kitchen...",Wall Street analysts suggested the worst was s...,2025-10-26 13:30:00+00:00,https://www.marketwatch.com/story/target-might...
3,general,CNBC,These friends were fired from dream jobs decad...,Laura Brown and Kristin O'Neill are the co-aut...,2025-10-26 13:45:01+00:00,https://www.cnbc.com/2025/10/26/how-to-survive...
4,general,CNBC,Here are the 3 big things we're watching in th...,"Microsoft, Apple and Meta are among the heavyw...",2025-10-26 14:47:09+00:00,https://www.cnbc.com/2025/10/26/here-are-the-3...


In [3]:
def finnhub_news_sentiment(symbol="AAPL"):
    data = client.news_sentiment(symbol)
    # returns aggregates like buzz, sentiment score, articles count
    return pd.json_normalize(data)

finnhub_news_sentiment("AAPL")


FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.